In [12]:
!pip install -q langchain faiss-cpu unstructured PyPDF2
!pip install -q huggingface_hub
!pip install -U langchain-community langchain-huggingface
!pip install -q langchain-huggingface
!pip install transformers datasets tqdm ddgs
!pip install langchain-embedding

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 7.1 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.
langchain-huggingface 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-community 0.4.1 requires langchain-core<2.0.0,>=1.0.1, but you have langchain-core 0.3.79 which is incompatible.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached langchain_core-1.0.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached langchain_text_splitters-1.0.0-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain_core-1.0.1-py3-none-any.whl (467 kB)
Using cached langchain_text_splitters-1.0.0-py3-none-any.whl (33 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.1 which is incompatible.
langchain 0.3.27

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement langchain-embedding (from versions: none)
ERROR: No matching distribution found for langchain-embedding


In [11]:
import os
import json
from tqdm import tqdm
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

2025-10-27 19:30:55.592773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761593455.916774      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761593456.006644      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# PDF Embedding

In [12]:
pdf_folder = "/kaggle/input/investing-books-pdf"  # change path if needed
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

In [13]:
all_documents = []

for pdf_path in pdf_files:
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()  # this returns list of Document objects
    all_documents.extend(docs)

print(f"Loaded {len(all_documents)} documents from PDFs.")

Loaded 4382 documents from PDFs.


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from typing import List

def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=350
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Created {len(split_docs)} text chunks.")
    return split_docs


# Example documents
docs = [
    Document(page_content="This is the content of document 1."),
    Document(page_content="This is the content of document 2.")
]

chunks = split_text(docs)
print(chunks[0].page_content)  # first chunk

Created 2 text chunks.
This is the content of document 1.


In [15]:
# 3) Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,   
    chunk_overlap=350  
)

split_docs = text_splitter.split_documents(all_documents)
print(f"Created {len(split_docs)} text chunks.")

Created 8126 text chunks.


In [16]:
split_docs[159].page_content

'John\'s z-score of –0.21 is higher than Ali\'s z-score of –0.3. For GPA, higher values are better, so we conclude that John has\nthe better GPA when compared to his school.\nTRY IT 2.32\nTwo swimmers, Angie and Beth, from different teams, wanted to find out who had the fastest time for the 50 meter\nfreestyle when compared to her team. Which swimmer had the fastest time when compared to her team?\nSwimmer Time (seconds) Team mean time Team standard deviation\nAngie 26.2 27.2 0.8\nBeth 27.3 30.1 1.4\nTable2.36\nThe following lists give a few facts that provide a little more insight into what the standard deviation tells us about the\ndistribution of the data.\nFor ANY data set, no matter what the distribution of the data is:\n• At least 75% of the data is within two standard deviations of the mean.\n• At least 89% of the data is within three standard deviations of the mean.\n• At least 95% of the data is within 4.5 standard deviations of the mean.\n• This is known as Chebyshev\'s Rule.

In [2]:
!pip install InstructorEmbedding

In [3]:
from InstructorEmbedding import INSTRUCTOR
import os

def get_embeddings(texts, model_path="/kaggle/input/qwen-3-embedding/transformers/4b/1", batch_size=8):
    # Path to your local embedding model
    model_path = "/kaggle/input/qwen-3-embedding/transformers/4b/1"
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    
    # Load the local embedding model
    embeddings = INSTRUCTOR(model_path, device=device)

    vector_list = embeddings_model.encode(texts, batch_size=batch_size)
    
    return vector_list

# Example usage:
texts = ["This is a sample text.", "Another example sentence."]
vectors = get_embeddings(texts)
print(vectors[0].shape)

/usr/local/lib/python3.11/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
2025-10-27 20:34:58.739123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761597299.042802      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761597299.126265      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to th

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(2560,)


In [7]:
vector_list

array([[-0.02290262,  2.7810962 ,  4.3660984 , ..., -1.4585104 ,
        -1.3493048 ,  1.0481972 ],
       [-0.02347616,  2.7970245 ,  4.2245073 , ...,  0.99478084,
        -2.4863873 ,  2.1848664 ]], dtype=float32)

In [ ]:
# 5) Build FAISS vector store
# -------------------------------
vector_db = FAISS.from_documents(split_docs, embeddings)

# Web Search

In [15]:
from ddgs import DDGS
from langchain.schema import Document
import requests
from bs4 import BeautifulSoup

def web_search(query, max_results=3):
    docs = []

    # Search the web using DuckDuckGo
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=max_results))

    for result in results:
        url = result.get("href")
        title = result.get("title")

        try:
            # Fetch the web page content
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            # Parse visible text from HTML
            soup = BeautifulSoup(response.text, "html.parser")
            for tag in soup(["script", "style", "noscript"]):
                tag.extract()
            text = " ".join(soup.get_text(separator=" ").split())

            # Only keep if text length is reasonable
            if len(text) > 500:
                docs.append(Document(page_content=text, metadata={"source": url, "title": title}))
        except Exception as e:
            # print(f"⚠️ Skipping {url}: {e}")

    # print(f"✅ Retrieved {len(docs)} web documents from top {max_results} results.")
    return docs

query = "Tesla 2025 annual revenue analysis"
documents = web_search(query)

print(f"Number of documents: {len(documents)}")
print(documents[0].metadata)
print(documents[0].page_content[:500])  # preview text

✅ Retrieved 3 web documents from top 3 results.
Number of documents: 3
{'source': 'https://stockanalysis.com/stocks/tsla/revenue/', 'title': 'Tesla (TSLA) Revenue 2015-2025'}
Tesla (TSLA) Revenue 2015-2025 Skip to main content Log In Sign Up Home Watchlist Stocks Stock Screener Stock Exchanges Comparison Tool Earnings Calendar By Industry Stock Lists Top Analysts Top Stocks Corporate Actions IPOs Recent IPOs IPO Calendar IPO Statistics IPO News IPO Screener ETFs ETF Screener Comparison Tool New Launches ETF Providers News Trending Articles Technical Chart Market Movers Top Gainers Top Losers Most Active Premarket After Hours Market Heatmap Market Newsletter Stock Ana


# Web PDF Retriever

In [ ]:
from ddgs import DDGS

def search_pdfs(company_name, year=2024, max_results=5):
    query = f"{company_name} {year} annual report filetype:pdf"
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=max_results))
    pdf_links = [r['href'] for r in results if r['href'].endswith('.pdf')]
    return pdf_links

# Example:
pdf_urls = search_pdfs("Square Pharma", 2024)
print(pdf_urls)

In [ ]:
import requests

def download_pdf(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200 and 'application/pdf' in response.headers.get('Content-Type', ''):
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        print(f"✅ Downloaded: {save_path}")
    else:
        print("❌ Not a valid PDF or download failed.")
# if pdf_urls:
    # for i in range(len(pdf_urls)):
        # download_pdf(pdf_urls[4], "apple_2024_annual_report.pdf")
download_pdf("https://www.squarepharma.com.bd/downloads/Square%20Pharma_AR_24%20dt-24-11-24_compressed_1.pdf", "hjd.pdf")

# Chain Construction

In [ ]:
from langchain.schema import Document
from langchain.runnables import (
    RunnablePassthrough,
    RunnableLambda,
    RunnableSequence,
    RunnableParallel,
)

# 1️⃣ User query passthrough
user_input = RunnablePassthrough()  # receives query from user

# 2️⃣ Online text search branch
online_text_chain = RunnableSequence(
    [
        RunnableLambda(lambda query: web_search(query)),  # returns raw text from web
        RunnableLambda(lambda text: split_text(text)),            # split into chunks
        RunnableLambda(lambda chunks: get_embeddings(chunks)),       # embeddings
    ]
)

# 3️⃣ Online PDF retrieval branch
online_pdf_chain = RunnableSequence(
    [
        RunnableLambda(lambda query: get_pdf_links(query)),       # get online PDF URLs
        RunnableLambda(lambda urls: download_pdfs(urls)),        # download PDFs
        RunnableLambda(lambda pdf_texts: split_text(pdf_texts)), # split text
        RunnableLambda(lambda chunks: get_embeddings(chunks)),       # embeddings
    ]
)

# 4️⃣ Preloaded investing book PDFs branch
book_pdf_chain = RunnableSequence(
    [
        RunnablePassthrough(),                 # preloaded PDF Documents passed in
        RunnableLambda(lambda docs: split_text(docs)),  # split into chunks
        RunnableLambda(lambda chunks: get_embeddings(chunks)),  # embeddings
    ]
)

# 5️⃣ Parallel execution of the three branches
parallel_chain = RunnableParallel(
    {
        "books": book_pdf_chain,
        "online_text": online_text_chain,
        "online_pdf": online_pdf_chain
    },
    combine_mode="list",  # returns a list of embeddings for each branch
)

# 6️⃣ Combine embeddings and build retrieval chain
combine_chain = RunnableSequence(
    [
        RunnableLambda(lambda embedding_lists: combine_embeddings(embedding_lists)),  # merge all embeddings
        RunnableLambda(lambda vector_db: rag_retrieval(vector_db)),                   # retrieval + augmentation
        RunnableLambda(lambda result: generate_answer(result)),                        # LLM generation
    ]
)

# 7️⃣ Full pipeline: sequence of parallel + combine
full_pipeline = RunnableSequence([parallel_chain, combine_chain])

# Example usage:
# preloaded_books = [...]  # list of Document objects
# query = "Analyze Tesla 2024 competition and market behavior"
# output = full_pipeline.invoke([preloaded_books, query, query])
